In [1]:
import os
os.getcwd()
os.chdir("H:\pyspark_advanced-coding_interview")
os.getcwd()

'H:\\pyspark_advanced-coding_interview'

# find the employee with the closest salary to the average salary within each department

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, abs, col, lit, row_number
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("ClosestSalary").getOrCreate()

# Sample Data
data = [
    (1, "Alice", 5000, "HR"),
    (2, "Bob", 6000, "HR"),
    (3, "Charlie", 5500, "HR"),
    (4, "David", 9000, "Engineering"),
    (5, "Eva", 8500, "Engineering"),
    (6, "Frank", 9500, "Engineering")
]

# Create DataFrame
df = spark.createDataFrame(data, ["emp_id", "emp_name", "salary", "department"])
df.cache()
df.createOrReplaceTempView("employees")



In [7]:
df.show()

+------+--------+------+-----------+
|emp_id|emp_name|salary| department|
+------+--------+------+-----------+
|     1|   Alice|  5000|         HR|
|     2|     Bob|  6000|         HR|
|     3| Charlie|  5500|         HR|
|     4|   David|  9000|Engineering|
|     5|     Eva|  8500|Engineering|
|     6|   Frank|  9500|Engineering|
+------+--------+------+-----------+



In [2]:

df = spark.createDataFrame(data, ["emp_id", "emp_name", "salary", "department"])
df.cache()
df.createOrReplaceTempView("employees")


res = spark.sql("""                 
                
 WITH dept_avg AS (
    SELECT department, AVG(salary) AS avg_salary
    FROM employees
    GROUP BY department
),
emp_with_avg AS (
    SELECT e.*, d.avg_salary, ABS(e.salary - d.avg_salary) AS diff
    FROM employees e
    JOIN dept_avg d ON e.department = d.department
),
ranked_employees AS (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY department ORDER BY diff) AS rank
    FROM emp_with_avg
)
SELECT emp_id, emp_name, salary, department
FROM ranked_employees
WHERE rank = 1;
               
                
                """)
res.show()

NameError: name 'spark' is not defined

In [3]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, abs, col, lit, row_number
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("ClosestSalary").getOrCreate()

# Sample Data
data = [
    (1, "Alice", 5000, "HR"),
    (2, "Bob", 6000, "HR"),
    (3, "Charlie", 5500, "HR"),
    (4, "David", 9000, "Engineering"),
    (5, "Eva", 8500, "Engineering"),
    (6, "Frank", 9500, "Engineering")
]

# Create DataFrame
df = spark.createDataFrame(data, ["emp_id", "emp_name", "salary", "department"])
df.createOrReplaceTempView("employees")





from pyspark.sql import Window
from pyspark.sql.functions import avg, abs, col, row_number

# Calculate average salary per department
avg_salary_df = df.groupBy("department").agg(avg("salary").alias("avg_salary"))

# Join with original data and calculate the difference
joined_df = df.join(avg_salary_df, "department") \
    .withColumn("diff", abs(col("salary") - col("avg_salary")))

# Define window to rank employees based on the closest difference
window_spec = Window.partitionBy("department").orderBy("diff")

# Get the employee with the closest salary to the average salary
result_df = joined_df.withColumn("rank", row_number().over(window_spec)) \
    .filter(col("rank") == 1) \
    .select("emp_id", "emp_name", "salary", "department")

result_df.show()


+------+--------+------+-----------+
|emp_id|emp_name|salary| department|
+------+--------+------+-----------+
|     4|   David|  9000|Engineering|
|     3| Charlie|  5500|         HR|
+------+--------+------+-----------+

